In [6]:
import os
import warnings; warnings.filterwarnings("ignore")
import torch as th
from crewai import (Agent, Task, Crew)
from langchain.llms import Ollama
from crewai_tools import SerperDevTool

In [2]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch version = {th.__version__}")
print(f"cuda version = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch version = 2.0.1+cu118
cuda version = 11.8


## step-1: 配置默认大模型（OpenAI）

In [3]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"  # gpt-4

## step-1: 配置本地大模型（Ollama）

In [3]:
ollama_openhermes = Ollama(model="openhermes")
ollama_solar = Ollama(model="Solar")

## step-2: Agent

In [4]:
agent = Agent(
    role="Data Analyst",
    goal="Extract actionable insights",
    backstory=(
       "You're a data analyst at a large company."
       "You're responsible for analyzing data and providing insights to the business."
       "You're currently working on a project to analyze the performance of our marketing campaigns."
   ),
    llm=ollama_openhermes,  # Optional, defaulting to "gpt-4" if not specified
    # tools=[my_tool1, my_tool2],  # Optional, defaults to an empty list
    allow_delegation=False,  # Optional, Default is True.
    verbose=True,  # Optional
    # cache=True  # Optional, Default is True.
    # function_calling_llm=my_llm,  # Optional
    # max_iter=25,  # Optional
    # max_rpm=None, # Optional
    # step_callback=my_intermediate_step_callback,  # Optional
)

In [8]:
research_agent = Agent(
    role="Researcher",
    goal="Find and summarize the latest AI news",
    backstory=(
        "You're a researcher at a large company."
        "You're responsible for analyzing data and providing insights to the business."
    ),
    llm=ollama_openhermes,
    allow_delegation=False,
    verbose=True
)

## step-3: Task

### 确保每个任务只设置一种输出类型，以保持清晰的输出

In [ ]:
search_tool = SerperDevTool()

In [11]:
task = Task(
  description="Find and summarize the latest AI news",
  expected_output="A bullet list summary of the top 5 most important AI news",
  agent=research_agent,
  tools=[search_tool],
#   context=[task_2, task_3],
#   async_execution=False,
#   output_json=class,
#   output_file="xxx.json",
#   human_input=False
)

In [14]:
research_ai_task = Task(
    description="Find and summarize the latest AI news",
    expected_output="A bullet list summary of the top 5 most important AI news",
    agent=research_agent,
    async_execution=True,
    tools=[search_tool]
)

In [15]:
research_ops_task = Task(
    description="Find and summarize the latest AI news",
    expected_output="A bullet list summary of the top 5 most important AI news",
    agent=research_agent,
    async_execution=True,
    tools=[search_tool]
)

In [ ]:
write_blog_task = Task(
    description="Write a full blog post about the importance of AI and its latest news",
    expected_output="Full blog post that is 4 paragraphs long",
    agent=writer_agent,
    context=[research_ai_task, research_ops_task]
)

In [ ]:
# The callback function is executed after the task is completed
def callback_function(output):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(f"""
        Task completed!
        Task: {output.description}
        Output: {output.raw_output}
    """)

research_task = Task(
    description="Find and summarize the latest AI news",
    expected_output="A bullet list summary of the top 5 most important AI news",
    agent=research_agent,
    tools=[search_tool],
    callback=callback_function
)

In [12]:
crew = Crew(
    agents=[research_agent],
    tasks=[research_ai_task, research_ops_task, write_blog_task],
    verbose=2
)

In [ ]:
result = crew.kickoff()
print(result)

In [ ]:
print(
    "Task completed!"
    f"Task: {research_ai_task.output.description}"
    f"Output: {research_ai_task.output.raw_output}"
)

## step-4: Tools